In [3]:
%load_ext autoreload
%autoreload 2
from simuq import QSystem
from simuq import Qubit
from simuq.ibm import IBMProvider
import numpy as np
ibm = IBMProvider(
    from_file="../../../qiskit_APIKEY", hub="ibm-q-ornl", group="ornl", project="phy147"
)

def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [Qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    if p==3:    
        parameter_list = np.array(
        [
            0.5702193 * 2,
            -0.58631086,
            0.85160685 * 2,
            -1.7058538,
            0.29468536 * 2,
            -1.132814,
        ]
    )/2
    if p==2:
        parameter_list = np.array([-3.8034,1.2438, -1.2467,-2.4899])/2
    if p==1:
        parameter_list = np.array([-0.7854,-2.3562])/2

    for i in range(p):
       
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].X * ql[q1].X
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i+1] * ql[q0].Z
        qs.add_evolution(h, 1)
    return qs

def calc_cut(count):
    cut=0
    n_shots=sum(count.values())
    for key in count:
        cut+=count[key]*calc_weight(key)/n_shots
    return cut
def calc_weight(key):
    cut=0
    for i in range(len(key)):
        if key[i]!= key[(i+1)%len(key)]:
            cut+=1
    return cut



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
for p in [1,2,3]:
    qs=GenQS(12,1)    
    ibm.compile(qs, backend="ibmq_guadalupe", trotter_num=4, use_pulse=False)
    ibm.run(on_simulator=True)
    ibm.results(on_simulator=True) 